<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Select-events" data-toc-modified-id="Select-events-0.1">Select events</a></span><ul class="toc-item"><li><span><a href="#Save-selected-events-to-file" data-toc-modified-id="Save-selected-events-to-file-0.1.1">Save selected events to file</a></span></li></ul></li><li><span><a href="#Align-clusters" data-toc-modified-id="Align-clusters-0.2">Align clusters</a></span></li><li><span><a href="#Add-tracks-and-vertex-to-events" data-toc-modified-id="Add-tracks-and-vertex-to-events-0.3">Add tracks and vertex to events</a></span><ul class="toc-item"><li><span><a href="#Save-tracked-events-to-file" data-toc-modified-id="Save-tracked-events-to-file-0.3.1">Save tracked events to file</a></span></li><li><span><a href="#Opening-angle" data-toc-modified-id="Opening-angle-0.3.2">Opening angle</a></span></li><li><span><a href="#DCA-between-two-tracks" data-toc-modified-id="DCA-between-two-tracks-0.3.3">DCA between two tracks</a></span></li><li><span><a href="#DCA-between-vertex-and-[0,0,0]" data-toc-modified-id="DCA-between-vertex-and-[0,0,0]-0.3.4">DCA between vertex and [0,0,0]</a></span></li></ul></li></ul></li><li><span><a href="#Very-clean-events" data-toc-modified-id="Very-clean-events-1">Very clean events</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Vertex-positions" data-toc-modified-id="Vertex-positions-1.0.1">Vertex positions</a></span></li><li><span><a href="#Position-shift-between-layers" data-toc-modified-id="Position-shift-between-layers-1.0.2">Position shift between layers</a></span></li><li><span><a href="#Correlation-between-ALPIDEs-after-all-selections" data-toc-modified-id="Correlation-between-ALPIDEs-after-all-selections-1.0.3">Correlation between ALPIDEs after all selections</a></span></li></ul></li></ul></li></ul></div>

In [12]:
import math
import matplotlib.pyplot as plt
import ROOT
import copy
import numpy as np
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from multiprocessing import Process
from multiprocessing import Pool
import time
import math

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

### Select events (and prealign azimuth)

In [ ]:
#first weekend files

#path1 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run447162516_221106162754.pkl" #200 MeV,14.9Gb, threshold not mentioned Run 57
#path2 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run445203522_221104212842.pkl" #120 MeV, 2.3Gb, threshold not mentioned Run 031, may be junk
#path3 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run446172943_221105174846.pkl" #80 MeV,  2.2Gb, threshold not mentioned Run 38


#second weekend (cleaner to use 200MeV)
path = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl" #200MeV 100e
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456103757_221112104316.pkl"  #120MeV 300e 
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl" #80MeV 100e 


#run456114514_221112120005.pkl"
  #uits3_krakow22/output/output.pkl" #yzAlign_run456114514_221112120005.pkl" #"/home/cehrich/Software/uits3_krakow22/run457063715_221113063928_clusters.pkl" #"/home/cehrich/Software/uits3_krakow22/run456195948_221112200957.pkl" 
#first file is 80MeV, 0.6 GB, second and third are 200 MeV at 3GB and 12GB respectively (pkl size)
nEvents = -1
selectedEvents = []
if nEvents > 0 :
    f = IntProgress(min=0, max=nEvents, description="Processing:")
    display(f)

shift = -1  #azimuth shift -1° but not doing that this time
needsFlip=True

for i,event in enumerate(readEvents(path, nEvents=nEvents)):
    #print("test")
    if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
    eventStatus = True
    event.clusters = [cluster.flipYaxis() for cluster in event.clusters]
    #event.clusters = [cluster.shiftXaxis("ALPIDE_1",shift, "angle") for cluster in event.clusters if 23 > cluster.size > 3]
        
    for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        clusters = event.selectDetector(alpide)
        if len(clusters) != 1:
            eventStatus = False
            break
    if eventStatus : selectedEvents.append(event)
        
if 'f' in locals(): 
    f.bar_style = "success"
    print("Found events:",len(selectedEvents),", Fraction:",len(selectedEvents)*100/nEvents,"%")

#### Save selected events to file

In [ ]:
path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents_brute.pkl"
save2pickle(selectedEvents,path_selected)

### Add tracks and vertex to events

In [ ]:
#path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl" #this was defined already above
events= []
for event in readEvents("/home/cehrich/Software/uits3_krakow22/selectedEvents_brute.pkl", nEvents=-1):
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    events.append(event)

#### Save tracked events to file

In [ ]:
path_tracked = "/home/cehrich/Software/uits3_krakow22/trackedEvents_brute.pkl"
save2pickle(events,path_tracked)

## Very clean events

In [ ]:
verySelectedEvents = []
firstEvents = 0

xMin=-2
xMax=2

yMin=-5
yMax=5

zMin=-20
zMax=-17

for event in loadEvents("/home/cehrich/Software/uits3_krakow22/trackedEvents_brute.pkl"): # events: 
    if 85 < event.vertex.openingAngle*360/(2*math.pi) < 90 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([7<cluster.size<20 for cluster in event.clusters]): #2sigma of 80MeV 100e target sizes
        if(all([(-8 < cluster.localPos[0] < 8 and -3 < cluster.localPos[1] < 3) for cluster in event.clusters])): #cuts out multiple scattering with plastic window
            if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                verySelectedEvents.append(event)
                if False: #(firstEvents <= 1):
                    print("\nEvent ", firstEvents)#,"\n")
                    for cluster in event.clusters:
                        print("Detector: ", cluster.detector, "Global position: ", cluster.globalPos, "Local position: ", cluster.localPos, "size:", cluster.size)#, "\n")
                    firstEvents+=1


In [ ]:
print(len(verySelectedEvents))
path_clean = "/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl"
save2pickle(verySelectedEvents,path_clean)

# Scan alignment

In [24]:
verySelectedEvents = loadEvents(path_clean)
len(verySelectedEvents)

616

In [9]:
import copy
%jsroot on
hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",200,-5,5,200,-23,-15)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
myVerySelectedEvents=copy.deepcopy(verySelectedEvents)
for event in myVerySelectedEvents:
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1])
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ")
canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")
canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")
canvas.Draw()

In [10]:
%jsroot on
hRMS = ROOT.TH1F("RMS cluster","RMS Cluster",22*5,-0.1,2.1)
hRMS.GetXaxis().SetTitle("RMS(cluster distance in track) [mm]")
hRMS.GetYaxis().SetTitle("Count")
for event in verySelectedEvents:
    for track in event.tracks:
        if track.nClusters == 3:
            hRMS.Fill(track.rms)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hRMS.Draw("")
canvas.Draw()


In [11]:
#%%capture cap --no-stderr
def evalDisplacement(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    
    
    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(np.mean(dcaVals))[:5], " and DCA sigma: ", str(np.std(dcaVals))[:5])
    return [medianDCA, medianTrackRMS]


ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 
def findBest(ranges,step):
    print("Round",step)
    uniqueCornArrs = []
    for [x1,x2,y1,y2] in ranges:
        corners = [[x,y,0] for x in [x1,x2] for y in [y1,y2]] 
        uniqueCorners = set(tuple(row) for row in corners)
        uniqueCornArr = [list(row) for row in uniqueCorners]
        uniqueCornArrs.append(uniqueCornArr)
    
    print(uniqueCornArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 corner alpides)(displacement 0 to 2) displacements (corner neglected )
    displacements = [{
        "ALPIDE_0" : corner0,
        "ALPIDE_1" : corner1,
        "ALPIDE_2" : corner2,
        "ALPIDE_3" : corner3,
        "ALPIDE_4" : corner4, 
    } for corner0 in uniqueCornArrs[0] for corner1 in uniqueCornArrs[1] for corner2 in uniqueCornArrs[2] for corner3 in uniqueCornArrs[3] for corner4 in uniqueCornArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    
    
    results = [evalDisplacement(displacement)[0] for displacement in displacements] # list of all the DCA values
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])[:10]) #switched [i] with [winner] since index out of bound
    
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
    else :
        findBest([[displacements[winner][ALPIDEs[i]][0] ,(x1+x2)/2,displacements[winner][ALPIDEs[i]][1],(y1+y2)/2] for i in range(5)], step-1)
        

    #with open('/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStats.txt', 'w') as f:
        #f.write(str(cap))
    
    #print("Winner of round",step,"is",[displacements[i][winner] for i in range(5)],"with",str(results[winner])[:5]) #switched uniqueCornArrs[i][winner][1] for displacements [i][winner][1] here since index out of bound
    
    #if step <= 0 :
    #    return [results[winner],[displacements[i][winner] for i in range(5)]]
    #else :
    #    findBest([[displacements[i][winner][0] ,(x1+x2)/2,displacements[i][winner][1],(y1+y2)/2] for i in range(5)], step-1)
    
    
    #print("Winner of round",step,"is",[uniqueCornArrs[i][winner] for i in range(5)],"with",str(results[winner])[:5])
    
    #if step <= 0 :
    #    return [results[winner],[uniqueCornArrs[i][winner] for i in range(5)]]
    #else :
    #    findBest([[uniqueCornArrs[i][winner][0] ,(x1+x2)/2,uniqueCornArrs[i][winner][1],(y1+y2)/2] for i in range(5)], step-1)
        

In [12]:
findBest([[0.2,0.5,-0.2,0.1],[0.4,0.65,-0.2,0.1],[0.55,0.75,-0.25,0],[0.2,0.5,-0.1,0.1],[0.55,0.85,-0.2,0.1]],6)

%%capture cap --no-stderr


with open('/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStatsMoreRefined_Without_Mult_Scattering.txt', 'w') as f:
        f.write(str(cap))

Round 6
[[[0.2, -0.2, 0], [0.5, 0.1, 0], [0.2, 0.1, 0], [0.5, -0.2, 0]], [[0.65, 0.1, 0], [0.4, -0.2, 0], [0.65, -0.2, 0], [0.4, 0.1, 0]], [[0.55, 0, 0], [0.55, -0.25, 0], [0.75, -0.25, 0], [0.75, 0, 0]], [[0.5, 0.1, 0], [0.2, -0.1, 0], [0.2, 0.1, 0], [0.5, -0.1, 0]], [[0.85, 0.1, 0], [0.55, 0.1, 0], [0.85, -0.2, 0], [0.55, -0.2, 0]]]
Going to try 1024 displacement configurations.
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.55, 0, 0], 'ALPIDE_3': [0.5, 0.1, 0], 'ALPIDE_4': [0.85, 0.1, 0]} . Got median DCA  0.31026743 . Got median track RMS  0.48285208  and mean DCA:  0.315  and DCA sigma:  0.125
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.55, 0, 0], 'ALPIDE_3': [0.5, 0.1, 0], 'ALPIDE_4': [0.55, 0.1, 0]} . Got median DCA  0.75058311 . Got median track RMS  0.48285208  and mean DCA:  0.754  and DCA sigma:  0.130
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 

Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.55, 0.1, 0]} . Got median DCA  0.73921543 . Got median track RMS  0.48285208  and mean DCA:  0.742  and DCA sigma:  0.129
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.85, -0.2, 0]} . Got median DCA  0.30063273 . Got median track RMS  0.48285208  and mean DCA:  0.305  and DCA sigma:  0.122
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.55, -0.2, 0]} . Got median DCA  0.73438687 . Got median track RMS  0.48285208  and mean DCA:  0.737  and DCA sigma:  0.128
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.75, -0.25, 0], 'ALPIDE_3': [0.5, 0.1, 0], 'ALPIDE_4': [0.85, 0.1, 0]} . Got me

Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.55, -0.2, 0]} . Got median DCA  0.48261041 . Got median track RMS  0.48285208  and mean DCA:  0.486  and DCA sigma:  0.124
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.85, 0.1, 0]} . Got median DCA  0.77247167 . Got median track RMS  0.48285208  and mean DCA:  0.775  and DCA sigma:  0.134
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.55, 0.1, 0]} . Got median DCA  1.21840780 . Got median track RMS  0.48285208  and mean DCA:  1.221  and DCA sigma:  0.134
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.65, 0.1, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.85, -0.2, 0]} . Got median DCA  0.7748

Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.85, -0.2, 0]} . Got median DCA  0.48252793 . Got median track RMS  0.48285208  and mean DCA:  0.486  and DCA sigma:  0.106
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.55, -0.2, 0]} . Got median DCA  0.07172575 . Got median track RMS  0.48285208  and mean DCA:  0.087  and DCA sigma:  0.107
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.85, 0.1, 0]} . Got median DCA  0.24033846 . Got median track RMS  0.48285208  and mean DCA:  0.246  and DCA sigma:  0.123
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.55, -0.25, 0], 'ALPIDE_3': [0.5, -0.1, 0], 'ALPIDE_4': [0.55, 0.1, 0]} . Got med

Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.85, 0.1, 0]} . Got median DCA  0.05842166 . Got median track RMS  0.48285208  and mean DCA:  0.076  and DCA sigma:  0.108
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.55, 0.1, 0]} . Got median DCA  0.42818915 . Got median track RMS  0.48285208  and mean DCA:  0.431  and DCA sigma:  0.126
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.85, -0.2, 0]} . Got median DCA  0.05923143 . Got median track RMS  0.48285208  and mean DCA:  0.075  and DCA sigma:  0.105
Evaluated displacement {'ALPIDE_0': [0.2, -0.2, 0], 'ALPIDE_1': [0.4, -0.2, 0], 'ALPIDE_2': [0.75, 0, 0], 'ALPIDE_3': [0.2, 0.1, 0], 'ALPIDE_4': [0.55, -0.2, 0]} . Got median DCA  0.4314806

...

Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.625, -0.10625, 0], 'ALPIDE_2': [0.75, -0.10625, 0], 'ALPIDE_3': [0.5546875, -0.10625, 0], 'ALPIDE_4': [0.55, -0.1109375, 0]} . Got median DCA  0.75861353 . Got median track RMS  0.48285208  and mean DCA:  0.762  and DCA sigma:  0.130
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.625, -0.10625, 0], 'ALPIDE_2': [0.75, -0.10625, 0], 'ALPIDE_3': [0.5546875, -0.10625, 0], 'ALPIDE_4': [0.55, -0.115625, 0]} . Got median DCA  0.75849474 . Got median track RMS  0.48285208  and mean DCA:  0.762  and DCA sigma:  0.130
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.625, -0.10625, 0], 'ALPIDE_2': [0.75, -0.10625, 0], 'ALPIDE_3': [0.5546875, -0.1109375, 0], 'ALPIDE_4': [0.5546875, -0.1109375, 0]} . Got median DCA  0.75214196 . Got median track RMS  0.48285208  and mean DCA:  0.756  and DCA sigma:  0.130
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625,

Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.625, -0.10625, 0], 'ALPIDE_2': [0.5546875, -0.1109375, 0], 'ALPIDE_3': [0.2, -0.10625, 0], 'ALPIDE_4': [0.5546875, -0.1109375, 0]} . Got median DCA  0.57756572 . Got median track RMS  0.48285208  and mean DCA:  0.578  and DCA sigma:  0.102
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.625, -0.10625, 0], 'ALPIDE_2': [0.5546875, -0.1109375, 0], 'ALPIDE_3': [0.2, -0.10625, 0], 'ALPIDE_4': [0.5546875, -0.115625, 0]} . Got median DCA  0.57765529 . Got median track RMS  0.48285208  and mean DCA:  0.578  and DCA sigma:  0.102
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.625, -0.10625, 0], 'ALPIDE_2': [0.5546875, -0.1109375, 0], 'ALPIDE_3': [0.2, -0.10625, 0], 'ALPIDE_4': [0.55, -0.1109375, 0]} . Got median DCA  0.57068339 . Got median track RMS  0.48285208  and mean DCA:  0.571  and DCA sigma:  0.102
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.1

Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.5546875, -0.1109375, 0], 'ALPIDE_2': [0.75, -0.1109375, 0], 'ALPIDE_3': [0.5546875, -0.1109375, 0], 'ALPIDE_4': [0.55, -0.1109375, 0]} . Got median DCA  0.73451986 . Got median track RMS  0.48285208  and mean DCA:  0.739  and DCA sigma:  0.130
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.5546875, -0.1109375, 0], 'ALPIDE_2': [0.75, -0.1109375, 0], 'ALPIDE_3': [0.5546875, -0.1109375, 0], 'ALPIDE_4': [0.55, -0.115625, 0]} . Got median DCA  0.73438656 . Got median track RMS  0.48285208  and mean DCA:  0.738  and DCA sigma:  0.130
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.5546875, -0.1109375, 0], 'ALPIDE_2': [0.75, -0.10625, 0], 'ALPIDE_3': [0.2, -0.1109375, 0], 'ALPIDE_4': [0.5546875, -0.1109375, 0]} . Got median DCA  0.14188133 . Got median track RMS  0.48285208  and mean DCA:  0.146  and DCA sigma:  0.117
Evaluated displacement {'ALPIDE_0': [0

Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.5546875, -0.1109375, 0], 'ALPIDE_2': [0.5546875, -0.10625, 0], 'ALPIDE_3': [0.5546875, -0.10625, 0], 'ALPIDE_4': [0.5546875, -0.1109375, 0]} . Got median DCA  0.27083241 . Got median track RMS  0.48285208  and mean DCA:  0.275  and DCA sigma:  0.125
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.5546875, -0.1109375, 0], 'ALPIDE_2': [0.5546875, -0.10625, 0], 'ALPIDE_3': [0.5546875, -0.10625, 0], 'ALPIDE_4': [0.5546875, -0.115625, 0]} . Got median DCA  0.27063437 . Got median track RMS  0.48285208  and mean DCA:  0.275  and DCA sigma:  0.125
Evaluated displacement {'ALPIDE_0': [0.5546875, -0.115625, 0], 'ALPIDE_1': [0.5546875, -0.1109375, 0], 'ALPIDE_2': [0.5546875, -0.10625, 0], 'ALPIDE_3': [0.5546875, -0.10625, 0], 'ALPIDE_4': [0.55, -0.1109375, 0]} . Got median DCA  0.27763523 . Got median track RMS  0.48285208  and mean DCA:  0.282  and DCA sigma:  0.125
Evaluated displacement

UsageError: Line magic function `%%capture` not found.


In [ ]:
#from IPython.utils.io import Tee
#with open('/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStatsMoreRefined.txt', 'a') as f, Tee(f, channel='stdout'):
#    output = findBest([[0,0.6,-0.5,0],[0,0.6,-0.5,0],[0,0.6,-0.5,0],[0,0.5,-0.5,0],[0,0.7,-0.5,0]],6)
#    print(output)

In [ ]:
#from IPython.utils.io import Tee
#from io import StringIO
#import sys
#
## Specify the file path for saving the output
#output_file = '/home/cehrich/Software/AlignmentOutput/Alignment_200MeV_100eMoreStatsMoreRefined.txt'
#
## Create a StringIO object to capture the printed output
#output_buffer = StringIO()
#
## Open the file in append mode and redirect the stdout to both the file and the StringIO object
#with open(output_file, 'a') as f, Tee(f, channel='stdout'), Tee(output_buffer, channel='stdout'):
#    # Redirect the stderr to the StringIO object to avoid capturing it in the file
#    sys.stderr = output_buffer
#
#    # Call the findBest function and print its output
#    findBest([[0.2,0.5,-0.2,0.1],[0.4,0.65,-0.2,0.1],[0.55,0.75,-0.25,0],[0.2,0.5,-0.1,0.1],[0.55,0.85,-0.2,0.1]],6)
#
#
## Retrieve the output from the StringIO object and print it in the notebook
#output_buffer.seek(0)
#output = output_buffer.read()
#print(output)

In [13]:
#preAlign


best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
} #Got median DCA  0.05772230 . Got median track RMS  0.48295208  and mean DCA:  0.082  and DCA sigma:  0.130
        

prealigned_events= []
for event in verySelectedEvents:
    for cluster in event.clusters:
        cluster.flipYaxis()
        cluster.alignLocal(best_displacement_no_multiple_scat.get(cluster.detector)) #displacements 6 has no displacement
    prealigned_events.append(event)

In [14]:
f = open('twistAlignmentrefined.out', 'w') 
 
print('this goes to the screen, or Jupyter notebook') 
 
print('Begin file', file=f) 
 


def evalDisplacementTwist(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(prealigned_events) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    
    
    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(np.mean(dcaVals))[:5], " and DCA sigma: ", str(np.std(dcaVals))[:5], file=f)
    return [medianDCA, medianTrackRMS]


ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 
def findBestTwist(ranges,step):
    print("Round",step)
    uniqueAngArrs = []
    for [alpha1,alpha2] in ranges:
        angles = [alpha for alpha in [alpha1, alpha2]] 
        #uniqueAngles = set(tuple(row) for row in angles)
        #uniqueAngArr = [list(row) for row in uniqueAngles]
        uniqueAngArrs.append(angles) #uniqueAngArr)
    
    print(uniqueAngArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 angle alpides)(displacement 0 to 2) displacements (angle neglected )
    #print(angles)
    displacements = [{
        "ALPIDE_0" : angle0,
        "ALPIDE_1" : angle1,
        "ALPIDE_2" : angle2,
        "ALPIDE_3" : angle3,
        "ALPIDE_4" : angle4, 
    } for angle0 in uniqueAngArrs[0] for angle1 in uniqueAngArrs[1] for angle2 in uniqueAngArrs[2] for angle3 in uniqueAngArrs[3] for angle4 in uniqueAngArrs[4]]  #for angle0 in angles[0] for angle1 in angles[1] for angle2 in angles[2] for angle3 in angles[3] for angle4 in angles[4]]
     
    print("Going to try",len(displacements),"displacement configurations.")
    
    
    #results = [evalDisplacementTwist(displacement)[0] for displacement in displacements] # list of all the DCA values
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalDisplacementTwist, displacements)
        print("Program finished!")
    
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])) #switched [i] with [winner] since index out of bound
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner]), file=f) #switched [i] with [winner] since index out of bound

    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
        f.close()
    else :
        findBestTwist([[displacements[winner][ALPIDEs[i]] ,(alpha1+alpha2)/2] for i in range(5)], step-1)
               

this goes to the screen, or Jupyter notebook


In [ ]:
findBestTwist([[0,0.5],[0.1,0.6],[0,0.5],[-0.3,0.3],[0,0.5]],20)

## Explore 15D Space

In [16]:
f = open('/home/cehrich/Software/AlignmentOutput/15DalignmentOne_Tenth_Angle.out', 'w') 
 
print('this goes to the screen, or Jupyter notebook') 
 
print('this goes to the file', file=f) 
 
      
        
def evalDisplacement15D(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:2] ) for alpide in ["ALPIDE_3","ALPIDE_4"]]) #corners
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)[3]) for alpide in ["ALPIDE_3","ALPIDE_4"]])  #angles
        
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:2]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) #corners
        TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)[3]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) #angles
        
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
                
    DCAnoTail = [dca <= np.median(dcaVals)+ np.std(dcaVals) for dca in dcaVals]
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    meanDCA = np.mean(dcaVals)
    quality = meanDCA + medianTrackRMS/2

    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(meanDCA)[:5], " and DCA sigma x,y,z: ", str(np.std(dcaVals[0]))[:5],", ", str(np.std(dcaVals[1]))[:5],", ", str(np.std(dcaVals[2]))[:5], file=f)

    return [meanDCA, medianDCA, medianTrackRMS, quality]


    
def tryDisplacements(displacementsArgs):
    #result.append(evalDisplacement15D(displacement)[0] for displacement in displacementsArgs)
    
    theseResults=[]
    result="" 
    for displacement in displacementsArgs:
        result=evalDisplacement15D(displacement)[3] # provides the weighted value of the fit
    theseResults.append(result)    
    return theseResults

ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 

def findBest15D(ranges,step):
    print("Round",step)
    uniqueCornArrs = []
    for [x1,x2,y1,y2,alpha1,alpha2] in ranges:
        corners = [[x,y,0,alpha] for x in [x1,x2] for y in [y1,y2] for alpha in [alpha1, alpha2]]
        #angles = [alpha for alpha in [alpha1,alpha2]] 
        
        uniqueCorners = set(tuple(row) for row in corners)
        uniqueCornArr = [list(row) for row in uniqueCorners]
        uniqueCornArrs.append(uniqueCornArr)
        
       # uniqueAngArrs.append(angles)
        
    print(uniqueCornArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 corner alpides)(displacement 0 to 2) displacements (corner neglected )
    displacements = [{
        "ALPIDE_0" : corner0,
        "ALPIDE_1" : corner1,
        "ALPIDE_2" : corner2,
        "ALPIDE_3" : corner3,
        "ALPIDE_4" : corner4, 
    } for corner0 in uniqueCornArrs[0] for corner1 in uniqueCornArrs[1] for corner2 in uniqueCornArrs[2] for corner3 in uniqueCornArrs[3] for corner4 in uniqueCornArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    print("Going to try",len(displacements),"displacement configurations.", file=f)
    #results = [evalDisplacement15D(displacement)[0] for displacement in displacements] # list of all the DCA values
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalDisplacement15D, displacements)
        print("Program finished!")
    
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])) #switched [i] with [winner] since index out of bound
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner]), file=f) 
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
        f.close()    
    else :
        findBest15D([[displacements[winner][ALPIDEs[i]][0] ,(x1+x2)/2,displacements[winner][ALPIDEs[i]][1],(y1+y2)/2,displacements[winner][ALPIDEs[i]][3],(alpha1+alpha2)/2] for i in range(5)], step-1)
        

 

this goes to the screen, or Jupyter notebook


In [18]:
findBest15D([[-0.3,0.3,-0.3, 0.3, -0.1, 0.1],[-0.3,0.3,-0.3, 0.3, -0.1, 0.1],[-0.3,0.3,-0.3, 0.3, -0.1, 0.1],[-0.3,0.3,-0.3, 0.3, -0.1, 0.1],[-0.3,0.3,-0.3, 0.3, -0.1, 0.1]],6)

Round 6
[[[0.3, -0.3, 0, 0.1], [-0.3, 0.3, 0, -0.1], [0.3, 0.3, 0, 0.1], [-0.3, -0.3, 0, -0.1], [-0.3, 0.3, 0, 0.1], [0.3, -0.3, 0, -0.1], [0.3, 0.3, 0, -0.1], [-0.3, -0.3, 0, 0.1]], [[0.3, -0.3, 0, 0.1], [-0.3, 0.3, 0, -0.1], [0.3, 0.3, 0, 0.1], [-0.3, -0.3, 0, -0.1], [-0.3, 0.3, 0, 0.1], [0.3, -0.3, 0, -0.1], [0.3, 0.3, 0, -0.1], [-0.3, -0.3, 0, 0.1]], [[0.3, -0.3, 0, 0.1], [-0.3, 0.3, 0, -0.1], [0.3, 0.3, 0, 0.1], [-0.3, -0.3, 0, -0.1], [-0.3, 0.3, 0, 0.1], [0.3, -0.3, 0, -0.1], [0.3, 0.3, 0, -0.1], [-0.3, -0.3, 0, 0.1]], [[0.3, -0.3, 0, 0.1], [-0.3, 0.3, 0, -0.1], [0.3, 0.3, 0, 0.1], [-0.3, -0.3, 0, -0.1], [-0.3, 0.3, 0, 0.1], [0.3, -0.3, 0, -0.1], [0.3, 0.3, 0, -0.1], [-0.3, -0.3, 0, 0.1]], [[0.3, -0.3, 0, 0.1], [-0.3, 0.3, 0, -0.1], [0.3, 0.3, 0, 0.1], [-0.3, -0.3, 0, -0.1], [-0.3, 0.3, 0, 0.1], [0.3, -0.3, 0, -0.1], [0.3, 0.3, 0, -0.1], [-0.3, -0.3, 0, 0.1]]]
Going to try 32768 displacement configurations.
Program finished!
Winner of round 6 is [[0.3, -0.3, 0, -0.1], [0.3, -0.3,

Program finished!
Winner of round 1 is [[0.3, 0.0, 0, -0.1], [0.3, -0.22499999999999998, 0, -0.1], [0.3, -0.26249999999999996, 0, 0.1], [-0.3, 0.3, 0, 0.1], [-0.3, -0.3, 0, 0.1]] with [0.07744920295367486, 0.060942894086899985, 0.48285208975322036, 0.31887524783028504]
Round 0
[[[0.3, 0.0, 0, 0.096875], [0.3, -0.290625, 0, 0.096875], [-0.290625, -0.290625, 0, 0.096875], [0.3, 0.0, 0, -0.1], [0.3, -0.290625, 0, -0.1], [-0.290625, -0.290625, 0, -0.1], [-0.290625, 0.0, 0, 0.096875], [-0.290625, 0.0, 0, -0.1]], [[0.3, -0.290625, 0, 0.096875], [-0.290625, -0.290625, 0, 0.096875], [0.3, -0.290625, 0, -0.1], [-0.290625, -0.290625, 0, -0.1], [0.3, -0.22499999999999998, 0, 0.096875], [0.3, -0.22499999999999998, 0, -0.1], [-0.290625, -0.22499999999999998, 0, 0.096875], [-0.290625, -0.22499999999999998, 0, -0.1]], [[-0.290625, -0.290625, 0, 0.1], [0.3, -0.290625, 0, 0.096875], [-0.290625, -0.26249999999999996, 0, 0.1], [-0.290625, -0.290625, 0, 0.096875], [-0.290625, -0.26249999999999996, 0, 0.09

In [6]:
path_clean = "/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl"
verySelectedEvents = []
for event in readEvents(path_clean, nEvents=-1):
    verySelectedEvents.append(event)

    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    
    event.vertex = Vertex()
    event.vertex.fromTracks([TrackLeft,TrackRight])

Going to process all events.


In [9]:
f = open('/home/cehrich/Software/AlignmentOutput/15Dalignment_z_out', 'w') 
 
print('this goes to the screen, or Jupyter notebook') 
 
print('this goes to the file', file=f) 
 
      
        
def evalDisplacement15Dz(displacement):
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3] ) for alpide in ["ALPIDE_3","ALPIDE_4"]]) #corners
        
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) #corners
        
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
                
    DCAnoTail = [dca <= np.median(dcaVals)+ np.std(dcaVals) for dca in dcaVals]
    medianDCA = np.median(dcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    meanDCA = np.mean(dcaVals)
    quality = meanDCA + medianTrackRMS/2

    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(meanDCA)[:5], " and DCA sigma x,y,z: ", str(np.std(dcaVals[0]))[:5],", ", str(np.std(dcaVals[1]))[:5],", ", str(np.std(dcaVals[2]))[:5], file=f)

    return [meanDCA, medianDCA, medianTrackRMS, quality] #try subtracting the median openning angle from the actual openning angle additionally to assess quality


    
def tryDisplacementz(displacementsArgs):
    #result.append(evalDisplacement15D(displacement)[0] for displacement in displacementsArgs)
    
    theseResults=[]
    result="" 
    for displacement in displacementsArgs:
        result=evalDisplacement15Dz(displacement)[3] # provides the weighted value of the fit
    theseResults.append(result)    
    return theseResults

ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 

def findBest15Dz(ranges,step):
    print("Round",step)
    uniqueCornArrs = []
    for [x1,x2,y1,y2,z1,z2] in ranges:
        corners = [[x,y,z] for x in [x1,x2] for y in [y1,y2] for z in [z1, z2]]

        
        uniqueCorners = set(tuple(row) for row in corners)
        uniqueCornArr = [list(row) for row in uniqueCorners]
        uniqueCornArrs.append(uniqueCornArr)

        
    print(uniqueCornArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 corner alpides)(displacement 0 to 2) displacements (corner neglected )
    displacements = [{
        "ALPIDE_0" : corner0,
        "ALPIDE_1" : corner1,
        "ALPIDE_2" : corner2,
        "ALPIDE_3" : corner3,
        "ALPIDE_4" : corner4, 
    } for corner0 in uniqueCornArrs[0] for corner1 in uniqueCornArrs[1] for corner2 in uniqueCornArrs[2] for corner3 in uniqueCornArrs[3] for corner4 in uniqueCornArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    print("Going to try",len(displacements),"displacement configurations.", file=f)
    #results = [evalDisplacement15D(displacement)[0] for displacement in displacements] # list of all the DCA values
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalDisplacement15Dz, displacements)
        print("Program finished!")
    
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner])) #switched [i] with [winner] since index out of bound
    print("Winner of round",step,"is",[displacements[winner][ALPIDEs[i]] for i in range(5)],"with",str(results[winner]), file=f) 
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
        f.close()    
    else :
        findBest15Dz([[displacements[winner][ALPIDEs[i]][0] ,(x1+x2)/2,displacements[winner][ALPIDEs[i]][1],(y1+y2)/2,displacements[winner][ALPIDEs[i]][2],(z1+z2)/2] for i in range(5)], step-1)
        

 

this goes to the screen, or Jupyter notebook


In [ ]:
start_time = time.perf_counter()
findBest15Dz([[-0.3,0.3,-0.3, 0.3, -0.3, 0.3],[-0.3,0.3,-0.3, 0.3, -0.3, 0.3],[-0.3,0.3,-0.3, 0.3, -0.3, 0.3],[-0.3,0.3,-0.3, 0.3, -0.3, 0.3],[-0.3,0.3,-0.3, 0.3, -0.3, 0.3]],6)
end_time = time.perf_counter()

elapsed_time = (end_time - start_time) / (3600)

print(f"Alignment took {elapsed_time:.4f} hours to execute.")


In [39]:
%jsroot on
hVertexRoman_Plane = ROOT.TH2F("ALPIDE Displacement","ALPIDE Displacement",200,-2,2,200,-2,2)
hVertexRoman_Plane.GetXaxis().SetTitle("X Displacement [mm]")
hVertexRoman_Plane.GetYaxis().SetTitle("Y Displacement [mm]")

h1DA0 = ROOT.TH1F("ALPIDE 0 Displacement [mm]","ALPIDE Displacement [mm]",300,-0.5,0.5)
h1DA1 = ROOT.TH1F("ALPIDE 1 Displacement [mm]","ALPIDE 1 Displacement [mm]",300,-0.5,0.5)
h1DA2 = ROOT.TH1F("ALPIDE 2 Displacement [mm]","ALPIDE 2 Displacement [mm]",300,-0.5,0.5)
h1DA3 = ROOT.TH1F("ALPIDE 3 Displacement [mm]","ALPIDE 3 Displacement [mm]",300,-0.5,0.5)
h1DA4 = ROOT.TH1F("ALPIDE 4 Displacement [mm]","ALPIDE 4 Displacement [mm]",300,-0.5,0.5)
plots=[[],[],[],[],[],[]]
for j in range(6):
    plots[j] = copy.deepcopy([h1DA0,h1DA1,h1DA2,h1DA3,h1DA4])
colors=[ROOT.kRed, ROOT.kBlue, ROOT.kGreen, ROOT.kBlack, ROOT.kMagenta]
#colors=[1,2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("canvas", "Multiple Plots", 800, 600)
legend = ROOT.TLegend(0.6, 0.7, 0.9, 0.9)


    
    
ALPIDES=["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]

def distance_to_plane(point, plane_normal, plane_d):
    distance = (np.dot(point, plane_normal) + plane_d)/ np.linalg.norm(plane_normal)
    
    return distance
        
def romanDisplacement(displacement, DCA_calc,A1,A2,A3,plotNum):
    dist_arr0= []
    dist_arr1= []
    dist_arr2= []
    dist_arr3= []
    dist_arr4= []
    title_str="Plane formed by ALPIDE's: "+A1[7]+", " + A2[7]+", and " + A3[7]
    plots[plotNum][0].SetTitle(title_str)
    dist_arr = [dist_arr0,dist_arr1,dist_arr2,dist_arr3,dist_arr4]
    
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl")) #problem is that the events are changed in the algorithm so a copy must be used
    
    for event in tempEvents:
        if(DCA_calc):
            TrackLeft = Track()
            TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3] ) for alpide in ["ALPIDE_3","ALPIDE_4"]])
            
            TrackRight = Track()
            TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) 
            
            vertex = Vertex()
            vertex.fromTracks([TrackLeft,TrackRight])
            dcaVals.append(vertex.dca)
        for alpide in ALPIDES:
            event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:5])
        v1 = np.array(event.selectDetector(A1)[0].globalPos)- np.array(event.selectDetector(A2)[0].globalPos)
        v2 = np.array(event.selectDetector(A1)[0].globalPos)- np.array(event.selectDetector(A3)[0].globalPos)
        
        norm = [v1[1] * v2[2] - v1[2] * v2[1],
                v1[2] * v2[0] - v1[0] * v2[2],
                v1[0] * v2[1] - v1[1] * v2[0]]
        
        point = np.array(event.selectDetector(A1)[0].globalPos)
        
        D = -(norm[0] * point[0] + norm[1] * point[1] + norm[2] * point[2])
        dist=[distance_to_plane(np.array(event.selectDetector(alpide)[0].globalPos), norm, D) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]]

        
        for i, plot in enumerate(plots[plotNum]):
            plot.Fill(dist[i])
            dist_arr[i].append(dist[i])
            #if
            #legend.AddEntry(plot, "ALPIDE " + str(i), "l")

        #hVertexRoman_Plane.Fill(dist1,dist2)
        
        
        

    if(DCA_calc):   
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
        DCAnoTail = [dca <= np.median(dcaVals)+ np.std(dcaVals) for dca in dcaVals]
        medianDCA = np.median(dcaVals)
        medianTrackRMS = np.median(trackRMSVals)
        meanDCA = np.mean(dcaVals)
        quality = meanDCA + medianTrackRMS/2
    
        print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(meanDCA)[:5], " and DCA sigma x,y,z: ", str(np.std(dcaVals[0]))[:5],", ", str(np.std(dcaVals[1]))[:5],", ", str(np.std(dcaVals[2]))[:5], file=f)
    
        return [meanDCA, medianDCA, medianTrackRMS, quality] #try subtracting the median openning angle from the actual openning angle additionally to assess quality
    
    return [title_str, *(np.median(Adist) for Adist in dist_arr)]

null_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [0, 0, 0], 
     'ALPIDE_2': [0, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}
custom_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [1.1, 0, 0], 
     'ALPIDE_2': [0.2, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}

alignment01092023={
     'ALPIDE_0': [-0.28125, -0.225, 0.15], 
     'ALPIDE_1': [-0.225, -0.24375, 0.15], 
     'ALPIDE_2': [0.3, -0.24375, 0.075], 
     'ALPIDE_3': [-0.3, -0.2625, 0.075], 
     'ALPIDE_4': [-0.3, -0.24375, 0.15]
}

best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
}

berkin_ml_displacement={
    'ALPIDE_0': [-0.20006977021694183, -0.8966014385223389,0], 
    'ALPIDE_1': [0.7433865666389465, -0.8977992534637451,0], 
    'ALPIDE_2': [-0.8155627250671387, -0.9625105261802673,0], 
    'ALPIDE_3': [0.10266361385583878, 0.31308597326278687,0], 
    'ALPIDE_4': [0.4219255745410919, 1.322750449180603,0]
}
temp_x_align={
    'ALPIDE_0': [0, -1.75, 0], 
    'ALPIDE_1': [1.1, 1.296875, 0], 
    'ALPIDE_2': [0.2, 1.0, 0], 
    'ALPIDE_3': [0, 1.75, 0], 
    'ALPIDE_4': [0, -1.25, 0]
}
displacement=temp_x_align
print(*romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_4",0))
print(*romanDisplacement(displacement, False, "ALPIDE_1", "ALPIDE_2", "ALPIDE_4",1))
print(*romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_3",2))
print(*romanDisplacement(displacement, False, "ALPIDE_1", "ALPIDE_2", "ALPIDE_3",3))
print(*romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_3", "ALPIDE_4",4))
print(*romanDisplacement(displacement, False, "ALPIDE_2", "ALPIDE_3", "ALPIDE_4",5))
#Plane formed by str.get()
canvas.Divide(3,2)
legends = [copy.deepcopy(legend) for _ in range(6)]
for j in range(6):
    #h1DA0.Draw("SAME")
    #h1DA1.Draw("SAME")
    #h1DA2.Draw("SAME")
    #h1DA3.Draw("SAME")
    #h1DA4.Draw("SAME")
    canvas.cd(j+1)
    for i,plot in enumerate(plots[j]): #we want a plot for all 6 of these
        plot.Draw("SAME")
        plot.SetLineColor(colors[i])#(colors[i])
        legend_entry=legends[j].AddEntry(plot, "ALPIDE " + str(i), "L")
        legend_entry.SetLineColor(colors[i])
        plot.SetStats(0)
    legends[j].Draw()
canvas.Update()
canvas.Draw()
 

Plane formed by ALPIDE's: 0, 2, and 4 0.0 4.38604125689864e-05 0.0 -3.432726582853959e-05 0.0
Plane formed by ALPIDE's: 1, 2, and 4 -0.00010638830692223602 0.0 0.0 0.0015031403950972186 0.0
Plane formed by ALPIDE's: 0, 2, and 3 0.0 -8.138451720185997e-05 0.0 0.0 5.745045179373618e-05
Plane formed by ALPIDE's: 1, 2, and 3 0.00020918120951595975 0.0 0.0 0.0 -0.0027022369860050426
Plane formed by ALPIDE's: 0, 3, and 4 0.0 0.0003076282151196104 3.396988848131674e-05 0.0 0.0
Plane formed by ALPIDE's: 2, 3, and 4 -5.645572061459118e-05 0.0009744894556203541 0.0 0.0 0.0


Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE Displacement (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 0 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 1 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 2 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 3 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 4 Displacement [mm] (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


## 5D Local Y Displacement

In [3]:
def evalYDisplacement(displacement):
    dcaYVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaYVals.append(vertex.dcaVec[1])
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaYVals)
    medianTrackRMS = np.median(trackRMSVals)
    sigmaDCAY = np.std(dcaYVals)
    disp_sum=np.sum(np.abs(romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_3")))
    #disp_sum+=np.sum(romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_3"))
    #disp_sum+=np.sum(romanDisplacement(displacement, False, "ALPIDE_1", "ALPIDE_3", "ALPIDE_4"))

    print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean Y DCA: ", str(np.mean(dcaYVals))[:5], " and Y DCA sigma: ", str(sigmaDCAY)[:5])
    return (disp_sum) #[sigmaDCAY, medianTrackRMS] #maybe choose a better evaluation method ()


ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 
def findBestLocalY(ranges,xAlignment,step):
    print("Round",step)
    uniqueSideArrs = []
    for i, [y1,y2] in enumerate(ranges):
        sides = [[xAlignment[i],y,0] for y in [y1,y2]] 
        uniqueSides = set(tuple(row) for row in sides)
        uniqueSideArr = [list(row) for row in uniqueSides]
        uniqueSideArrs.append(uniqueSideArr)
    
    print(uniqueSideArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 side alpides)(displacement 0 to 2) displacements (side neglected )
    displacements = [{
        "ALPIDE_0" : side0,
        "ALPIDE_1" : side1,
        "ALPIDE_2" : side2,
        "ALPIDE_3" : side3,
        "ALPIDE_4" : side4, 
    } for side0 in uniqueSideArrs[0] for side1 in uniqueSideArrs[1] for side2 in uniqueSideArrs[2] for side3 in uniqueSideArrs[3] for side4 in uniqueSideArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalYDisplacement, displacements)
        print("Program finished!")
        
    #results = [evalYDisplacement(displacement)[0] for displacement in displacements] # list of all the DCA values
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",displacements[winner],"with",str(results[winner])[:10]) #switched [i] with [winner] since index out of bound
    
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
    else :
        findBestLocalY([[displacements[winner][ALPIDEs[i]][1],np.mean(ranges[i])] for i in range(5)], xAlignment, step-1)

In [250]:
start_time = time.perf_counter()

findBestLocalY([[-2.0,2.0],[-2.0,2.0],[-2.0,2.0],[-2.0,2.0],[-2.0,2.0]],[0,1.1,0.2,0,0],10)

end_time = time.perf_counter()

elapsed_time = (end_time - start_time) / (60)

print(f"Alignment took {elapsed_time:.4f} minutes to execute.")

Round 10
[[[0, 2.0, 0], [0, -2.0, 0]], [[1.1, -2.0, 0], [1.1, 2.0, 0]], [[0.2, 2.0, 0], [0.2, -2.0, 0]], [[0, 2.0, 0], [0, -2.0, 0]], [[0, 2.0, 0], [0, -2.0, 0]]]
Going to try 32 displacement configurations.
Evaluated displacement {'ALPIDE_0': [0, 2.0, 0], 'ALPIDE_1': [1.1, -2.0, 0], 'ALPIDE_2': [0.2, -2.0, 0], 'ALPIDE_3': [0, -2.0, 0], 'ALPIDE_4': [0, 2.0, 0]} Evaluated displacement. Got median DCA   {'ALPIDE_0': [0, 2.0, 0], 'ALPIDE_1': [1.1, -2.0, 0], 'ALPIDE_2': [0.2, 2.0, 0], 'ALPIDE_3': [0, 2.0, 0], 'ALPIDE_4': [0, 2.0, 0]}0.00388187 . Got median DCA   . Got median track RMS 0.02738525 0.48285208  and mean Y DCA:   . Got median track RMS 0.003  0.48285208 and Y DCA sigma: Evaluated displacement   0.126
 and mean Y DCA: {'ALPIDE_0': [0, 2.0, 0], 'ALPIDE_1': [1.1, -2.0, 0], 'ALPIDE_2': [0.2, -2.0, 0], 'ALPIDE_3': [0, 2.0, 0], 'ALPIDE_4': [0, 2.0, 0]}  0.032 . Got median DCA  and Y DCA sigma:   -0.20896850.129 
. Got median track RMS  0.48285208  and mean Y DCA:  -0.20  and Y DCA si

 {'ALPIDE_0': [0, -2.0, 0], 'ALPIDE_1': [1.1, 0.0, 0], 'ALPIDE_2': [0.2, 2.0, 0], 'ALPIDE_3': [0, 2.0, 0], 'ALPIDE_4': [0, 0.0, 0]} . Got median DCA  0.05501695 . Got median track RMS  0.48285208  and mean Y DCA: Evaluated displacement  0.059{'ALPIDE_0': [0, -2.0, 0], 'ALPIDE_1': [1.1, 0.0, 0], 'ALPIDE_2': [0.2, 0.0, 0], 'ALPIDE_3': [0, 2.0, 0], 'ALPIDE_4': [0, 0.0, 0]}  and Y DCA sigma:  . Got median DCA  -0.0190596  . Got median track RMS  0.154
0.48285208  and mean Y DCA:  -0.01  and Y DCA sigma:  0.126
Evaluated displacement {'ALPIDE_0': [0, -2.0, 0], 'ALPIDE_1': [1.1, 0.0, 0], 'ALPIDE_2': [0.2, 0.0, 0], 'ALPIDE_3': [0, 0.0, 0], 'ALPIDE_4': [0, 0.0, 0]} . Got median DCA  0.06567018 . Got median track RMS  0.48285208  and mean Y DCA:  0.068  and Y DCA sigma:  0.165
Evaluated displacement {'ALPIDE_0': [0, -2.0, 0], 'ALPIDE_1': [1.1, 0.0, 0], 'ALPIDE_2': [0.2, 2.0, 0], 'ALPIDE_3': [0, 0.0, 0], 'ALPIDE_4': [0, -2.0, 0]} Evaluated displacement . Got median DCA {'ALPIDE_0': [0, -2.0, 0],

Evaluated displacement {'ALPIDE_0': [0, -1.5, 0], 'ALPIDE_1': [1.1, 1.0, 0], 'ALPIDE_2': [0.2, 1.5, 0], 'ALPIDE_3': [0, 1.5, 0], 'ALPIDE_4': [0, -1.0, 0]}Evaluated displacement  {'ALPIDE_0': [0, -1.5, 0], 'ALPIDE_1': [1.1, 1.0, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.5, 0], 'ALPIDE_4': [0, -1.0, 0]} . Got median DCA . Got median DCA   0.00626415 0.02452846. Got median track RMS  . Got median track RMS  0.48285208 0.48285208  and mean Y DCA:   0.029 and mean Y DCA:   0.011 and Y DCA sigma:    and Y DCA sigma: 0.135
 0.128
Evaluated displacement {'ALPIDE_0': [0, -1.5, 0], 'ALPIDE_1': [1.1, 1.0, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 2.0, 0], 'ALPIDE_4': [0, -1.5, 0]} . Got median DCA  -0.0212720 . Got median track RMS  0.48285208  and mean Y DCA:  -0.01  and Y DCA sigma:  0.126
Evaluated displacement {'ALPIDE_0': [0, -1.5, 0], 'ALPIDE_1': [1.1, 1.0, 0], 'ALPIDE_2': [0.2, 1.5, 0], 'ALPIDE_3': [0, 2.0, 0], 'ALPIDE_4': [0, -1.5, 0]} . Got median DCA  Evaluated displacement 

Evaluated displacement {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.25, 0], 'ALPIDE_2': [0.2, 1.125, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, -1.25, 0]} Evaluated displacement. Got median DCA   {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.25, 0], 'ALPIDE_2': [0.2, 1.125, 0], 'ALPIDE_3': [0, 1.625, 0], 'ALPIDE_4': [0, -1.25, 0]}0.00407325  . Got median DCA  . Got median track RMS  0.00887507 . Got median track RMS 0.48285208  0.48285208 and mean Y DCA:   and mean Y DCA:   0.0130.009   and Y DCA sigma:  and Y DCA sigma:   0.1290.128

Evaluated displacement {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.25, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.625, 0], 'ALPIDE_4': [0, -1.375, 0]} . Got median DCA  0.00239373 . Got median track RMS  0.48285208  and mean Y DCA:  0.006  and Y DCA sigma: Evaluated displacement  0.127
{'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.25, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, -1.375, 0]} . Got median DCA 

Evaluated displacement Evaluated displacementEvaluated displacement {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.3125, 0], 'ALPIDE_2': [0.2, 1.03125, 0], 'ALPIDE_3': [0, 1.71875, 0], 'ALPIDE_4': [0, -1.28125, 0]} {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.3125, 0], 'ALPIDE_2': [0.2, 1.03125, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, -1.28125, 0]} {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.3125, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.71875, 0], 'ALPIDE_4': [0, -1.28125, 0]}  . Got median DCA . Got median DCA  0.00149009 . Got median DCA 0.00139584  . Got median track RMS  0.00256935 . Got median track RMS 0.48285208   . Got median track RMS  and mean Y DCA: 0.48285208  0.005 0.48285208  and Y DCA sigma:  and mean Y DCA:  0.005   and mean Y DCA:  0.127 and Y DCA sigma: 
  0.1270.006
  and Y DCA sigma:  0.127
Evaluated displacement {'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.3125, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, -1.25

Evaluated displacement {'ALPIDE_0': [0, -1.734375, 0], 'ALPIDE_1': [1.1, 1.296875, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.734375, 0], 'ALPIDE_4': [0, -1.265625, 0]} . Got median DCA  Evaluated displacement 0.00067251{'ALPIDE_0': [0, -1.734375, 0], 'ALPIDE_1': [1.1, 1.296875, 0], 'ALPIDE_2': [0.2, 1.015625, 0], 'ALPIDE_3': [0, 1.734375, 0], 'ALPIDE_4': [0, -1.265625, 0]}  . Got median track RMS . Got median DCA  0.48285208   and mean Y DCA: 0.00121362  . Got median track RMS  0.005  and Y DCA sigma: 0.48285208  0.127 and mean Y DCA: 
 0.005 Evaluated displacement  and Y DCA sigma: {'ALPIDE_0': [0, -1.734375, 0], 'ALPIDE_1': [1.1, 1.296875, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, -1.265625, 0]} 0.127 
. Got median DCA  8.55244093 . Got median track RMS  0.48285208  and mean Y DCA: Evaluated displacement  0.004{'ALPIDE_0': [0, -1.734375, 0], 'ALPIDE_1': [1.1, 1.296875, 0], 'ALPIDE_2': [0.2, 1.015625, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, 

Evaluated displacement {'ALPIDE_0': [0, -1.74609375, 0], 'ALPIDE_1': [1.1, 1.29296875, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.74609375, 0], 'ALPIDE_4': [0, -1.25, 0]} . Got median DCA  0.00087015Evaluated displacement  {'ALPIDE_0': [0, -1.74609375, 0], 'ALPIDE_1': [1.1, 1.29296875, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.7421875, 0], 'ALPIDE_4': [0, -1.25, 0]}. Got median track RMS   0.48285208. Got median DCA    and mean Y DCA: 0.00101698  0.005. Got median track RMS    and Y DCA sigma: 0.48285208   and mean Y DCA: 0.127
 0.005  and Y DCA sigma:  0.127
Evaluated displacement {'ALPIDE_0': [0, -1.74609375, 0], 'ALPIDE_1': [1.1, 1.29296875, 0], 'ALPIDE_2': [0.2, 1.00390625, 0], 'ALPIDE_3': [0, 1.7421875, 0], 'ALPIDE_4': [0, -1.25, 0]} . Got median DCA  0.00115221 . Got median track RMS  0.48285208  and mean Y DCA:  0.005Evaluated displacement  {'ALPIDE_0': [0, -1.74609375, 0], 'ALPIDE_1': [1.1, 1.29296875, 0], 'ALPIDE_2': [0.2, 1.00390625, 0], 'ALPIDE_3': [0, 1.7460937

## 5D Local X Displacement

In [4]:
def evalXDisplacement(displacement):
    dcaYVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(verySelectedEvents) #problem is that the events are changed in the algorithm so a copy must be used
    for event in tempEvents:#loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaYVals.append(vertex.dcaVec[1])
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
    medianDCA = np.median(dcaYVals)
    medianTrackRMS = np.median(trackRMSVals)
    sigmaDCAY = np.std(dcaYVals)
    disp_sum=np.sum(np.abs(romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_3"))) #I am not sure if we can decide the coordinates from the plane displacement 
    #disp_sum+=np.sum(np.abs(romanDisplacement(displacement, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_3")))#from the axis. this is in the local x plane but we were shrinking it with local y values. 
    #disp_sum+=np.sum(np.abs(romanDisplacement(displacement, False, "ALPIDE_1", "ALPIDE_3", "ALPIDE_4")))#Might be better to choose only one displacement at a time than trying to minimize all as well as the DCA spread
    
    print("Evaluated displacement" ,displacement ,". Got median global Y DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean global Y DCA: ", str(np.mean(dcaYVals))[:5], " and global Y DCA sigma: ", str(sigmaDCAY)[:5])
    return (disp_sum) #[sigmaDCAX, medianTrackRMS] #maybe choose a better evaluation method ()


ALPIDEs = ["ALPIDE_0","ALPIDE_1", "ALPIDE_2", "ALPIDE_3", "ALPIDE_4"] 
def findBestLocalX(ranges,yAlignment,step):
    print("Round",step)
    uniqueSideArrs = []
    for i, [x1,x2] in enumerate(ranges):
        sides = [[x,yAlignment[i],0] for x in [x1,x2]] 
        uniqueSides = set(tuple(row) for row in sides)
        uniqueSideArr = [list(row) for row in uniqueSides]
        uniqueSideArrs.append(uniqueSideArr)
    
    print(uniqueSideArrs)  #prints a list of 5 (winner 0 to 4 displacement sets)(0 to 4 -1 side alpides)(displacement 0 to 2) displacements (side neglected )
    displacements = [{
        "ALPIDE_0" : side0,
        "ALPIDE_1" : side1,
        "ALPIDE_2" : side2,
        "ALPIDE_3" : side3,
        "ALPIDE_4" : side4, 
    } for side0 in uniqueSideArrs[0] for side1 in uniqueSideArrs[1] for side2 in uniqueSideArrs[2] for side3 in uniqueSideArrs[3] for side4 in uniqueSideArrs[4]]
    
    print("Going to try",len(displacements),"displacement configurations.")
    
    results= None    
   
    if __name__ == "__main__":
        with Pool() as pool:
              results = pool.map(evalXDisplacement, displacements)
        print("Program finished!")
        
    #results = [evalXDisplacement(displacement)[0] for displacement in displacements] # list of all the DCA values
    
    winner = results.index(min(results)) #lowest index of the lowest median dca Value. returns a large number since the list is 1024 long I think. 
    
    winningDisplacement = results[winner]
    
    print("Winner of round",step,"is",displacements[winner],"with",str(results[winner])[:10]) #switched [i] with [winner] since index out of bound
    
    if step <= 0 :
        return [results[winner],[displacements[winner][ALPIDEs[i]] for i in range(5)]]
    else :
        findBestLocalX([[displacements[winner][ALPIDEs[i]][0],np.mean(ranges[i])] for i in range(5)], yAlignment, step-1)

In [246]:
start_time = time.perf_counter()

findBestLocalX([[-1.0,1.0],[-1.0,1.0],[-1.0,1.0],[-1.0,1.0],[.0,.0]],[ 0,0,0,0,0],7)
end_time = time.perf_counter()

elapsed_time = (end_time - start_time) / (60)

print(f"Alignment took {elapsed_time:.4f} minutes to execute.")

Round 7
[[[1.0, 0, 0], [-1.0, 0, 0]], [[1.0, 0, 0], [-1.0, 0, 0]], [[1.0, 0, 0], [-1.0, 0, 0]], [[1.0, 0, 0], [-1.0, 0, 0]], [[0.0, 0, 0]]]
Going to try 16 displacement configurations.
Evaluated displacement Evaluated displacementEvaluated displacement {'ALPIDE_0': [1.0, 0, 0], 'ALPIDE_1': [1.0, 0, 0], 'ALPIDE_2': [-1.0, 0, 0], 'ALPIDE_3': [-1.0, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} {'ALPIDE_0': [1.0, 0, 0], 'ALPIDE_1': [1.0, 0, 0], 'ALPIDE_2': [1.0, 0, 0], 'ALPIDE_3': [1.0, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} . Got median global Y DCA  {'ALPIDE_0': [1.0, 0, 0], 'ALPIDE_1': [1.0, 0, 0], 'ALPIDE_2': [1.0, 0, 0], 'ALPIDE_3': [-1.0, 0, 0], 'ALPIDE_4': [0.0, 0, 0]}  -6.7662202. Got median global Y DCA  . Got median global Y DCA  . Got median track RMS Evaluated displacement  -2.2559011 0.482852082.57608576   {'ALPIDE_0': [1.0, 0, 0], 'ALPIDE_1': [1.0, 0, 0], 'ALPIDE_2': [-1.0, 0, 0], 'ALPIDE_3': [1.0, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} and mean global Y DCA: . Got median track RMS . Got median track R

Evaluated displacement Evaluated displacement{'ALPIDE_0': [-0.25, 0, 0], 'ALPIDE_1': [1.0, 0, 0], 'ALPIDE_2': [0.25, 0, 0], 'ALPIDE_3': [-0.25, 0, 0], 'ALPIDE_4': [0.0, 0, 0]}  . Got median global Y DCA {'ALPIDE_0': [-0.25, 0, 0], 'ALPIDE_1': [1.0, 0, 0], 'ALPIDE_2': [0.5, 0, 0], 'ALPIDE_3': [-0.25, 0, 0], 'ALPIDE_4': [0.0, 0, 0]}  -0.0963285. Got median global Y DCA  . Got median track RMS   0.482852080.48356229  . Got median track RMS  and mean global Y DCA:  0.48285208  -0.09  and mean global Y DCA:  and global Y DCA sigma:   0.1260.485
  and global Y DCA sigma:  0.127
Evaluated displacementEvaluated displacement  {'ALPIDE_0': [-0.25, 0, 0], 'ALPIDE_1': [0.75, 0, 0], 'ALPIDE_2': [0.25, 0, 0], 'ALPIDE_3': [-0.5, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} {'ALPIDE_0': [-0.25, 0, 0], 'ALPIDE_1': [0.75, 0, 0], 'ALPIDE_2': [0.25, 0, 0], 'ALPIDE_3': [-0.25, 0, 0], 'ALPIDE_4': [0.0, 0, 0]}. Got median global Y DCA   . Got median global Y DCA -0.7843272  -0.1794022 . Got median track RMS  . Got median

Evaluated displacement {'ALPIDE_0': [-0.3125, 0, 0], 'ALPIDE_1': [0.96875, 0, 0], 'ALPIDE_2': [0.28125, 0, 0], 'ALPIDE_3': [-0.28125, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} . Got median global Y DCA  Evaluated displacement-0.0059045  {'ALPIDE_0': [-0.3125, 0, 0], 'ALPIDE_1': [0.96875, 0, 0], 'ALPIDE_2': [0.28125, 0, 0], 'ALPIDE_3': [-0.25, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} . Got median global Y DCA  0.06928420 . Got median track RMS  . Got median track RMS 0.48285208   and mean global Y DCA: 0.48285208  and mean global Y DCA:   0.071-0.00   and global Y DCA sigma:  and global Y DCA sigma:  0.127Evaluated displacement 0.127 

{'ALPIDE_0': [-0.3125, 0, 0], 'ALPIDE_1': [0.96875, 0, 0], 'ALPIDE_2': [0.25, 0, 0], 'ALPIDE_3': [-0.28125, 0, 0], 'ALPIDE_4': [0.0, 0, 0]} . Got median global Y DCA  -0.0775882 . Got median track RMS  0.48285208  and mean global Y DCA:  Evaluated displacement-0.07   and global Y DCA sigma: {'ALPIDE_0': [-0.3125, 0, 0], 'ALPIDE_1': [0.96875, 0, 0], 'ALPIDE_2': [0.25, 0, 0],